# Imports

In [1]:
from utils import *

# Loading Datasets

In [2]:
# path = '/Users/joshuawong/Desktop/dataset_1'
path = 'C:/Users/user/Documents/GitHub/Fake-News-Detection/Data' #lingjia

os.chdir(path)
os.listdir()

['all_data_preprocessed.csv',
 'all_data_preprocessed_w_lemmatization.csv',
 'all_data_preprocessed_w_lemmatization.zip',
 'dataset_1',
 'dataset_2',
 'test_data.csv',
 'train_data.csv',
 'train_data.zip',
 'validation_data.csv']

## Reading Dataset 1: Fake and real news from Kaggle 

In [8]:
# Reading CSV files
fake = pd.read_csv(path + '/dataset_1/Fake.csv')
true = pd.read_csv(path + '/dataset_1/True.csv')

# Size of each class
print("Number of fake articles:", len(fake))
print("Number of true articles:", len(true))

# Defing new column with class label
fake['class_label'] = 1
true['class_label'] = 0

Number of fake articles: 23481
Number of true articles: 21417


In [9]:
# Combine title and text columns
fake['text'] = fake['title'] + ' ' + fake['text']
true['text'] = true['title'] + ' ' + true['text']

# Drop date and title columns
fake = fake.drop(columns=['date', 'title', 'subject'])
true = true.drop(columns=['date', 'title', 'subject'])
fake.head(5)

,text,class_label
0,Donald Trump Sends Out Embarrassing New Year’...,1
1,Drunk Bragging Trump Staffer Started Russian ...,1
2,Sheriff David Clarke Becomes An Internet Joke...,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,1
4,Pope Francis Just Called Out Donald Trump Dur...,1


## Reading Dataset 2: 

In [10]:
data2= pd.read_csv(path + "/dataset_2/train.csv")
# data2_test = pd.read_csv(path + "/dataset_2/test.csv")

data2.head(1)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1


In [6]:
print("Size of dataset2:", len(data2))
data2["label"].value_counts()

Size of dataset2: 20800


1    10413
0    10387
Name: label, dtype: int64

In [11]:
data2["text"] = data2["title"] + data2["text"]

data2 = data2.drop(["id", "title", "author"], axis=1)
data2.head(5)

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You FiredWhy the Truth...,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [12]:
data2.rename(columns={"label": "class_label"}, inplace=True)

In [9]:
# for row in data2.text:
#     try:
#         tokenised = row.split(" ")
#         if len(tokenised) < 0:
#             print(tokenised)
#     except:
#         print(row)

In [10]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         20203 non-null  object
 1   class_label  20800 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 325.1+ KB


In [13]:
# Dropping all rows where article has no text 
data2.dropna(inplace=True)

len(data2)

20203

In [12]:
data2.head(5)

,text,class_label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You FiredWhy the Truth...,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


# Preprocessing

In [13]:
# Testing preprocessing function for wordclouds, which require a string as input
text = true['text'].iloc[0] # Declare sample text for preprocessing from df

print("Before preprocessing:")
print(text[:200]) # Uncomment to see original text
print("--------------------------------------------------------------------------------\n")

# Pre-processing steps
remove_punctuations = ''.join([word for word in text if word not in string.punctuation and word not in ['“', '”', '’', '‘']])
lowercase = remove_punctuations.lower()
tokenize = word_tokenize(lowercase)
stopwords_removed = [word for word in tokenize if word not in STOPWORDS]
stemmed = [PorterStemmer().stem(word) for word in stopwords_removed]
# lemmatized = [WordNetLemmatizer().lemmatize(word) for word in stopwords_removed]
preprocessed_text = " ".join(stemmed) # Join back all tokenized words into a string

print("After preprocessing:")
print(preprocessed_text[:200]) # Uncomment to see original text
# Once satisfied, make into function

Before preprocessing:
As U.S. budget fight looms, Republicans flip their fiscal script WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of
--------------------------------------------------------------------------------

After preprocessing:
us budget fight loom republican flip fiscal script washington reuter head conserv republican faction us congress vote month huge expans nation debt pay tax cut call fiscal conserv sunday urg budget re


In [14]:
def preprocess(text):
    remove_punctuations = ''.join([word for word in text if word not in string.punctuation and word not in ['“', '”', '’', '‘']])
    lowercase = remove_punctuations.lower()
    tokenized = word_tokenize(lowercase)
    stopwords_removed = [word for word in tokenized if word not in STOPWORDS]
    stemmed = [PorterStemmer().stem(word) for word in stopwords_removed]
    preprocessed_text = " ".join(stemmed)
    return preprocessed_text

In [14]:
# Preprocessing function to include lemmatization
def preprocess2(text):
    remove_punctuations = ''.join([word for word in text if word not in string.punctuation and word not in ['“', '”', '’', '‘']])
    lowercase = remove_punctuations.lower()
    tokenized = word_tokenize(lowercase)
    stopwords_removed = [word for word in tokenized if word not in STOPWORDS]
    stemmed = [PorterStemmer().stem(word) for word in stopwords_removed]
    lemmatized = [WordNetLemmatizer().lemmatize(word) for word in stemmed]
    preprocessed_text = " ".join(lemmatized)
    return preprocessed_text

In [16]:
# Apply function to text column and create new column for use later with wordclouds
fake['text_preprocessed'] = fake['text'].apply(preprocess)
true['text_preprocessed'] = true['text'].apply(preprocess)

In [17]:
data2['text_preprocessed'] = data2['text'].apply(preprocess)

In [21]:
data_all = pd.concat([fake, true, data2])
data_all['text_preprocessed']

0        donald trump send embarrass new year eve messa...
1        drunk brag trump staffer start russian collus ...
2        sheriff david clark becom internet joke threat...
3        trump ob even obama name code websit imag chri...
4        pope franci call donald trump christma speech ...
                               ...                        
20795    rapper ti trump poster child white supremacyra...
20796    nfl playoff schedul matchup odd new york times...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkansnato ...
20799    keep f35 aliv david swanson author activist jo...
Name: text_preprocessed, Length: 65101, dtype: object

In [22]:
data_all2.to_csv("all_data_preprocessed_w_lemmatization.csv")

## Combing both datasets together

In [18]:
data_all = pd.concat([fake, true, data2])
data_all.head()

,text,class_label,text_preprocessed
0,Donald Trump Sends Out Embarrassing New Year’...,1,donald trump send embarrass new year eve messa...
1,Drunk Bragging Trump Staffer Started Russian ...,1,drunk brag trump staffer start russian collus ...
2,Sheriff David Clarke Becomes An Internet Joke...,1,sheriff david clark becom internet joke threat...
3,Trump Is So Obsessed He Even Has Obama’s Name...,1,trump obsess even obama name code websit imag ...
4,Pope Francis Just Called Out Donald Trump Dur...,1,pope franci call donald trump christma speech ...


In [19]:
print(len(fake)+len(true)+len(data2))
print(len(data_all))

65101
65101


In [ ]:
data_all.to_csv("../Data/all_data_preprocessed.csv")

# Train-Validation-Test Split

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# Splitting the dataset to train and test subsets
train, test = train_test_split(data_all, test_size=0.20, stratify=data_all['class_label'], shuffle=True)

# Splitting train to training and validation subsets
train, val = train_test_split(train, test_size=0.25, stratify=train['class_label'], shuffle=True)

In [31]:
print("Ratio of class labels for training dataset:\n", train['class_label'].value_counts()[0]/train['class_label'].value_counts()[1], "\n", train['class_label'].value_counts())
print("------------------------------------")

print("Ratio of class labels for validation dataset:\n", val['class_label'].value_counts()[0]/val['class_label'].value_counts()[1], "\n", val['class_label'].value_counts())
print("------------------------------------")

print("Ratio of class labels for test dataset:\n", test['class_label'].value_counts()[0]/test['class_label'].value_counts()[1],"\n", test['class_label'].value_counts())


Ratio of class labels for training dataset:
 0.9551506657323056 
 1    19978
0    19082
Name: class_label, dtype: int64
------------------------------------
Ratio of class labels for validation dataset:
 0.9552485358161886 
 1    6659
0    6361
Name: class_label, dtype: int64
------------------------------------
Ratio of class labels for test dataset:
 0.9551051051051052 
 1    6660
0    6361
Name: class_label, dtype: int64


In [30]:
train.to_csv("../Data/train_data.csv")
val.to_csv("../Data/validation_data.csv")
test.to_csv("../Data/test_data.csv")